In [1]:
import pandas as pd
#from pandas.io.json import json_normalize
import json
from sqlalchemy import create_engine

In [2]:
json_file = json.load(open('Resources/NYC_Tree_Census_2015.json'))
print(len(json_file["meta"]["view"]["columns"]))

column_names = []

for i in range(len(json_file["meta"]["view"]["columns"])):
    column_names.append(json_file["meta"]["view"]["columns"][i]["name"])
    
print(column_names)

tree_census_df = pd.DataFrame(json_file["data"], columns = column_names)
tree_census_df.head()


50
['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'created_at', 'tree_id', 'block_id', 'the_geom', 'tree_dbh', 'stump_diam', 'curb_loc', 'status', 'health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'user_type', 'problems', 'root_stone', 'root_grate', 'root_other', 'trnk_wire', 'trnk_light', 'trnk_other', 'brnch_ligh', 'brnch_shoe', 'brnch_othe', 'address', 'zipcode', 'zip_city', 'cb_num', 'borocode', 'boroname', 'cncldist', 'st_assem', 'st_senate', 'nta', 'nta_name', 'boro_ct', 'state', 'Latitude', 'longitude', 'x_sp', 'y_sp']


,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,created_at,tree_id,...,st_assem,st_senate,nta,nta_name,boro_ct,state,Latitude,longitude,x_sp,y_sp
0,row-3pa8.vipt-y3tu,00000000-0000-0000-A637-5EE16F0AF5E5,0,1487359879,None,1487359879,None,{ },08/27/2015,180683,...,28,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749
1,row-ezzh_t2qa_5rxb,00000000-0000-0000-C736-7E72AB353F4E,0,1487359879,None,1487359879,None,{ },09/03/2015,200540,...,27,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379
2,row-qhkh-tsv3-8r8i,00000000-0000-0000-4A19-ED152BD35BAA,0,1487359879,None,1487359879,None,{ },09/05/2015,204026,...,50,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267
3,row-etk8~4g6g_a4bj,00000000-0000-0000-059F-9D28D83E528A,0,1487359879,None,1487359879,None,{ },09/05/2015,204337,...,53,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136
4,row-rp9a~wqm6~mnvk,00000000-0000-0000-EF5D-330C8A3DF5C8,0,1487359879,None,1487359879,None,{ },08/30/2015,189565,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999


In [3]:
tree_columns = ['tree_id', 'health', 'zipcode', 'boroname', 'address']
tree_transform = tree_census_df[tree_columns].copy()
tree_transform = tree_transform.rename(columns={'tree_id':'id',
                                                   'health':'tree_health',
                                                   'zipcode':'zipcode_id',
                                                   'boroname':'borough',
                                                   'address':'address'})
tree_transformed = tree_transform.dropna(how='any',axis=0)
tree_transformed.set_index('id', inplace=True)
tree_transformed.head()

,tree_health,zipcode_id,borough,address
id,,,,
180683,Fair,11375,Queens,108-005 70 AVENUE
200540,Fair,11357,Queens,147-074 7 AVENUE
204026,Good,11211,Brooklyn,390 MORGAN AVENUE
204337,Good,11211,Brooklyn,1027 GRAND STREET
189565,Good,11215,Brooklyn,603 6 STREET


In [4]:
connection_string = "postgres:postgres@localhost:5432/tree_db"
engine = create_engine(f'postgresql://{connection_string}')

In [5]:
#next confirm the tables
engine.table_names()

['tree_census']

In [6]:
#next load the dataframe into SQL
tree_transformed.to_sql(name='tree_census', con=engine, if_exists='append', index=True)